# Algorithm 3: Contact Conditioning (Boltz-2)

Guide structure predictions with experimental contact information.

## Source Code Location
- **File**: `Boltz-Ref-src/boltz-official/src/boltz/model/modules/trunkv2.py`
- **Class**: `ContactConditioning`

In [ ]:
import numpy as np
np.random.seed(42)

def layer_norm(x, eps=1e-5):
    mean = np.mean(x, axis=-1, keepdims=True)
    var = np.var(x, axis=-1, keepdims=True)
    return (x - mean) / np.sqrt(var + eps)

In [ ]:
def fourier_embedding(x, dim=64, max_period=10000):
    """Sinusoidal embedding."""
    half = dim // 2
    freqs = np.exp(-np.log(max_period) * np.arange(half) / half)
    args = x[..., np.newaxis] * freqs
    return np.concatenate([np.cos(args), np.sin(args)], axis=-1)

In [ ]:
def contact_conditioning(contact_info, contact_threshold, token_z=128, 
                         cutoff_min=2.0, cutoff_max=10.0):
    """
    Contact Conditioning.
    
    Encodes experimental contact constraints to guide predictions.
    
    Args:
        contact_info: Contact information [N, N, num_types]
            - Type 0: UNSPECIFIED
            - Type 1: UNSELECTED  
            - Type 2+: Specific contact types
        contact_threshold: Distance thresholds [N, N]
        token_z: Output dimension
        cutoff_min: Minimum distance cutoff
        cutoff_max: Maximum distance cutoff
    
    Returns:
        Contact conditioning features [N, N, token_z]
    """
    N = contact_info.shape[0]
    num_types = contact_info.shape[-1]
    
    print(f"Contact Conditioning")
    print(f"="*50)
    print(f"Pairs: {N}x{N}, Contact types: {num_types}")
    
    # Separate special types
    unspecified = contact_info[:, :, 0:1]
    unselected = contact_info[:, :, 1:2]
    specific_contacts = contact_info[:, :, 2:]
    
    # Normalize threshold
    threshold_norm = (contact_threshold - cutoff_min) / (cutoff_max - cutoff_min)
    threshold_norm = np.clip(threshold_norm, 0, 1)
    
    # Fourier embedding of threshold
    threshold_emb = fourier_embedding(threshold_norm, dim=token_z)
    
    # Concatenate features
    features = np.concatenate([
        specific_contacts,
        threshold_norm[:, :, np.newaxis],
        threshold_emb,
    ], axis=-1)
    
    # Project to token_z
    W = np.random.randn(features.shape[-1], token_z) * (features.shape[-1] ** -0.5)
    output = np.einsum('ijc,cd->ijd', features, W)
    
    # Apply special type masks
    encoding_unspecified = np.random.randn(token_z) * 0.01
    encoding_unselected = np.random.randn(token_z) * 0.01
    
    mask = 1 - (unspecified + unselected)
    output = (
        output * mask + 
        encoding_unspecified * unspecified + 
        encoding_unselected * unselected
    )
    
    print(f"Specified contacts: {(mask > 0.5).sum()}")
    print(f"Output: {output.shape}")
    
    return output

In [ ]:
# Test
print("Test: Contact Conditioning")
print("="*60)

N = 32
num_types = 5  # UNSPECIFIED, UNSELECTED, CONTACT, NO_CONTACT, etc.

# Simulate contact information
contact_info = np.zeros((N, N, num_types))
contact_info[:, :, 0] = 0.8  # Most pairs unspecified

# Add some specified contacts
for i in range(5):
    j = i + 10
    contact_info[i, j, 0] = 0  # Not unspecified
    contact_info[i, j, 2] = 1  # Is a contact
    contact_info[j, i, 0] = 0
    contact_info[j, i, 2] = 1

contact_threshold = np.ones((N, N)) * 8.0  # 8A default

output = contact_conditioning(contact_info, contact_threshold)
print(f"\nOutput finite: {np.isfinite(output).all()}")

## Key Insights

1. **Experimental Guidance**: Use crosslinking, NMR, or other contact data
2. **Flexible Input**: Supports multiple contact types
3. **Distance Thresholds**: Variable thresholds per contact
4. **Drug Discovery**: Guide docking with known binding site info